In [1]:
import pandas as pd
from sympy.combinatorics import Permutation, PermutationGroup

In [2]:
class Problem:
    def __init__(self, id_, start_state, goal_state, subproblem_type, wildcards):
        self.id = id_
        self.start_state = start_state
        self.goal_state = goal_state
        self.subproblem_type = subproblem_type
        self.wildcards = wildcards

In [3]:
from functools import reduce

class SubProblem:
    """
    Represents a subproblem of the original problem with a specific id
    """
    def __init__(self, permutation_to_move_name, name_to_permutation, generator_set, problem):
        self.permutation_to_move_name = permutation_to_move_name
        self.name_to_permutation = name_to_permutation
        self.generator_set = generator_set
        self.permutation_group = PermutationGroup(generator_set)
        print(self.permutation_group.base)
        self.problem = problem

    def solve(self):
        """
        Returns the permutation that transforms the start state to the goal state
        WARN: This function only works if each element in the string is unique AND wildcards == 0!!
        """
        goal_perm_list = self.find_unique_permutation()
        goal_permutation = Permutation(goal_perm_list)
        length, move_decomp = self.get_generator_decomp(goal_permutation)
        return (length, move_decomp)
        
    def get_generator_decomp(self, group_element):
        """
        Uses permutation_group to decompose g into a product of generators
        """
        permutation_decomp = self.permutation_group.generator_product(group_element, original=True)
        move_names = []
        for permutation in permutation_decomp:
            move_names.append(self.permutation_to_move_name[tuple(permutation)])
        return (len(move_names), ".".join(move_names))
    
    def find_unique_permutation(self):
        """
        Returns the permutation that transforms the start state to the goal state
        WARN: This function only works if each element in the string is unique AND wildcards == 0!!
        """
        goal_element_to_index = {}
        start_list = self.problem.start_state.split(";")
        goal_list = self.problem.goal_state.split(";")
        for i, element in enumerate(start_list):
            goal_element_to_index[element] = i
        permutation = []
        for element in goal_list:
            permutation.append(goal_element_to_index[element])
        print(permutation)
        return permutation

In [4]:
class ProblemWrapper:
    """
    Represents a subproblem of the original problem with a specific id
    """
    def __init__(self, puzzles_path, puzzle_info_path, submission_path):
        self.type_to_moves = pd.read_csv(puzzle_info_path)
        self.id_to_puzzle = pd.read_csv(puzzles_path)
        # allowed moves: dict: puzzle_type -> dict: move_name -> move_permutation
        # e.g. "2x2x2": {f1 : [0, 1, ...], f2 : {1, 2, ...}}
        self.allowed_moves = {}
        for _, row in self.type_to_moves.iterrows():
            self.allowed_moves[row['puzzle_type']] = eval(row['allowed_moves'])

    def solve_subproblem(self, id):
        print("Solving subproblem with id: ", id)
        subproblem_type = self.id_to_puzzle["puzzle_type"][id]
        start_state = self.id_to_puzzle["initial_state"][id]
        goal_state = self.id_to_puzzle["solution_state"][id]
        num_wildcards = self.id_to_puzzle["num_wildcards"][id]

        subproblem = Problem(id, start_state, goal_state, subproblem_type, num_wildcards)
        perm_to_name = self.get_permutation_to_name_from_puzzle_type(subproblem_type)
        name_to_perm = self.get_name_to_permutation_from_puzzle_type(subproblem_type)
        generator_set = self.get_generator_set_from_puzzle_type(subproblem_type)

        subproblem_to_solve = SubProblem(perm_to_name, name_to_perm, generator_set, subproblem)
        return subproblem_to_solve.solve()

    def apply_perm_to_string(self, perm, string):
        """
        Applies permutation to a string of the form "element1;element2;..."
        """
        element_list = string.split(";")
        permuted_element_list = []
        for i in perm:
            permuted_element_list.append(element_list[i])
        return ";".join(permuted_element_list)

    def parse_move_string_to_permutation(self, move_string, name_to_permutation):
        move_list = move_string.split(".")
        move_list = list(map(lambda x: Permutation(name_to_permutation[x]), move_list))
        state = Permutation([])
        for move in move_list:
            state = move * state
        return state

    def assert_permutation_equal(self, p1_str, p2_str, name_to_permutation):
        p1 = self.parse_move_string_to_permutation(p1_str, name_to_permutation)
        p2 = self.parse_move_string_to_permutation(p2_str, name_to_permutation)
        assert p1 == p2

    def get_permutation_to_name_from_puzzle_type(self, puzzle_type):
        """
        Returns dict: permutation (tuple) -> move_name (str) as specified in puzzle_info.csv
        """
        permutation_to_move_name = {}

        for move_name, move_permutation in self.allowed_moves[puzzle_type].items():
            perm = Permutation(move_permutation)
            perm_inv = perm**-1
            permutation_to_move_name[tuple(perm)] = move_name
            permutation_to_move_name[tuple(perm_inv)] = "-" + move_name 
        return permutation_to_move_name

    def get_name_to_permutation_from_puzzle_type(self, puzzle_type):
        """
        Returns dict: move_name (str) -> permutation (tuple) as specified in puzzle_info.csv
        """
        name_to_permutation = {}

        for move_name, move_permutation in self.allowed_moves[puzzle_type].items():
            perm = Permutation(move_permutation)
            perm_inv = perm**-1
            name_to_permutation[move_name] = tuple(perm)
            name_to_permutation["-" + move_name] = tuple(perm_inv)
        return name_to_permutation
    
    def get_generator_set_from_puzzle_type(self, puzzle_type):
        """
        Returns list of generators (Permutation as specified in csv for allowed moves)
        """
        generator_set = []
        for _, move_permutation in self.allowed_moves[puzzle_type].items():
            generator_set.append(Permutation(move_permutation))
        return generator_set

In [8]:
problem_wrapper = ProblemWrapper("../data/puzzles.csv", "../data/puzzle_info.csv", "../data/sample_submission.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/puzzle_info.csv'

In [6]:
for key in problem_wrapper.allowed_moves.keys():
    items = []
    for perm in problem_wrapper.allowed_moves[key].items():
        name, perm = perm
        perm_inv = Permutation(perm)**-1
        items.append((name, perm))
        items.append(("-" + name, list(perm_inv)))
    # if path does not exist, create file and write to it
    key = key.replace("/", "_")
    with open("./minkwitz_solving/perms-nim/groups/G_" + key, "w+") as f:
        # first line: size of a permutation
        f.write(str(len(items[0][1])) + "\n")
        # second line: number of permutations
        f.write(str(len(items)) + "\n")
        # second line: permutations listed in name: permutation 
        for item in items:
            f.write(item[0] + ": " + str(item[1]) + "\n")
    print("Wrote group for puzzle type: ", key)

NameError: name 'problem_wrapper' is not defined

In [7]:
length, sol = problem_wrapper.solve_subproblem(20)

print(length, sol)

NameError: name 'problem_wrapper' is not defined